In [1]:
from transformers import AutoTokenizer, EsmForMaskedLM
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [30]:
model_name = "facebook/esm2_t6_8M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = EsmForMaskedLM.from_pretrained(model_name)
device = torch.device('cuda')
model.to(device)
print(next(model.parameters()).is_cuda)
sequence_list =  pd.read_csv("/scratch/user/yxliu/dataset/landscape/ProteinGym/reference_files/DMS_substitutions.csv")
#print(sequence_list.head())
DMS_id = sequence_list.iloc[0]["DMS_id"]
sequence_wt = sequence_list.iloc[0]["target_seq"]
seq_len = sequence_list.iloc[0]["seq_len"]

input_ids = tokenizer.encode(sequence_wt, return_tensors="pt")
start_pos = 1
end_pos = seq_len
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
heat_map = np.zeros((20, end_pos - start_pos+1))
for position in tqdm(range(start_pos,end_pos+1)):
    masked_input_ids = input_ids.clone()
    masked_input_ids[0,position]=tokenizer.mask_token_id
    masked_input_ids = masked_input_ids.to(device)
    #print(masked_input_ids.is_cuda)
    #print(masked_input_ids.shape)
    with torch.no_grad():
        logits = model(masked_input_ids).logits
        #print(logits)
    probabilities = torch.nn.functional.softmax(logits[0,position],dim=0)
    log_probabilities = torch.log(probabilities)
    wt_residue = input_ids[0,position].item()
    log_prob_wt = log_probabilities[wt_residue].item()
    for i,amino_acid in enumerate(amino_acids):
        log_prob_mt = log_probabilities[tokenizer.convert_tokens_to_ids(amino_acid)].item()
        heat_map[i,position-start_pos] = log_prob_mt - log_prob_wt
        
#sequence = sequence_list.iloc[0]["mutated_sequence"]
#label = sequence_list.iloc[0]["mutant"]

#input_ids = tokenizer.e

Token indices sequence length is longer than the specified maximum sequence length for this model (3425 > 1024). Running this sequence through the model will result in indexing errors


True


100%|█████████████████████████████████████████████████████████████████████| 3423/3423 [02:57<00:00, 19.32it/s]


In [ ]:
#Scoring Deep Mutational Scans
#adopted from blog: https://huggingface.co/blog/AmelieSchreiber/mutation-scoring


In [5]:
torch.cuda.empty_cache()

from esm import Alphabet,pretrained

sequence_list =  pd.read_csv("/scratch/user/yxliu/dataset/landscape/ProteinGym/reference_files/DMS_substitutions.csv")
#print(sequence_list.head())
DMS_id = sequence_list.iloc[0]["DMS_id"]
sequence_wt = sequence_list.iloc[0]["target_seq"]
seq_len = sequence_list.iloc[0]["seq_len"]

model,alphabet = pretrained.esm2_t6_8M_UR50D()
repr_layers = [int("esm2_t6_8M_UR50D".split('_')[1][1:])]
device = torch.device('cuda')
model = model.to(device)
model.eval()
batch_converter = alphabet.get_batch_converter()
data = [("protein1",sequence_wt)]
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_tokens = batch_tokens.to(device)
with torch.no_grad():
    #print(model(batch_tokens,repr_layers, return_contacts=True))
    result = model(batch_tokens,repr_layers, return_contacts=True)
    token_probs = torch.log_softmax(result["logits"],dim=-1)
    #token_probs = torch.log_softmax(model(batch_tokens)["logits"], dim=-1)
    print(result.keys())

OutOfMemoryError: CUDA out of memory. Tried to allocate 896.00 MiB. GPU 0 has a total capacty of 39.39 GiB of which 411.25 MiB is free. Process 34916 has 854.00 MiB memory in use. Including non-PyTorch memory, this process has 38.14 GiB memory in use. Of the allocated memory 36.06 GiB is allocated by PyTorch, and 1.59 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF